In [5]:
import time
import selenium.webdriver as webdriver

driver = webdriver.Chrome('C://chromedriver/chromedriver.exe')
url_start = 'https://news.naver.com'
keywords = ['킥보드', '자전거']

def input_keyword(keyword):
    driver.switch_to.window(driver.window_handles[0])
    driver.get(url_start)

    driver.implicitly_wait(10)
    driver.find_element_by_xpath('//input[@class="text_index"]').send_keys(keyword)
    driver.implicitly_wait(10)
    driver.find_element_by_xpath('//button[@type="submit"]').click()
    driver.switch_to.window(driver.window_handles[1])                  # 기존의 탭은 1번임. 2번째 탭을 생성
    driver.find_elements_by_xpath('//a[@role="option"]')[1].click()    # 새롭게 생성된 2번째 탭 클릭
    driver.implicitly_wait(10)
    return driver

def make_file(keyword):
    output_file_name = 'naver_news_' + keyword + "_" + time.strftime("%y%m%d_%H%M%S") + '.txt'
    output_file = open(output_file_name, "w", encoding="utf-8")
    output_file.write("{}\t{}\t{}\t{}\n".format('페이지', '키워드', '제목', 'URL'))
    output_file.close()
    return output_file_name

'''
Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36  - 크롬
'''
def write_news(i, keyword, news_title_clean, news_url, output_file_name):
    print([i, keyword, news_title_clean, news_url])
    output_file = open(output_file_name, "a", encoding='utf-8')
    output_file.write("{}\t{}\t{}\t{}\n".format(i, keyword, news_title_clean, news_url))
    output_file.close()
    return

def crawl_news_selenium(driver, keyword, i, output_file_name):
    bodies = driver.find_elements_by_xpath('//ul[@class="list_news"]/li')
    for body in bodies:
        news_title_elm = body.find_elements_by_xpath('.//a[@class="news_tit"]')[0]
        news_title = news_title_elm.get_attribute('title')
        try:
            news_url_elm = body.find_elements_by_xpath('.//a[@class="info"]')[0]
            news_url = news_url_elm.get_attribute('href')

        except:
            news_url = ''
        news_title_clean = news_title.replace("\n", "").replace("\t","").replace("\r","").strip()
        write_news(i, keyword, news_title_clean, news_url, output_file_name)
        page_nav = driver.find_element_by_xpath('//div[@class="sc_page_inner"]')
        next_page = page_nav.find_element_by_link_text(str(int(i) + 1))
        next_page.click()
        driver.implicitly_wait(10)
        return

def fmain():
    for keyword in keywords:
        output_file_name = make_file(keyword)
        driver = input_keyword(keyword)

        for i in range(1,3):
            print(i)
            crawl_news_selenium(driver,keyword,i,output_file_name)
        driver.close()
fmain()